In [17]:
from pyspark.sql import SparkSession
import timeit 

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.131:7077") \
        .appName("Team7_code.ipynb")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.cores.max",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")
print("Started Session")

Started Session


In [2]:


def add(a, b):
    # commutative and associative!
    return a + b

rdd = spark_context.parallelize([1, 2, 3, 4, 5, 6], 3)

result = rdd\
            .filter(lambda x: x % 2 == 0)\
            .map(lambda x: x ** 2)\
            .reduce(add)

print(result)

56


In [18]:
def create_spark_dataframe(filename):
    df = spark_session.read.json('hdfs://192.168.2.131:9000/user/ubuntu/{}'.format(filename))
    return df

def create_joint_spark_dataframe(regex):
    


df = create_spark_dataframe('RC_2005-12.json')


df.printSchema()

root
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: long (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- edited: boolean (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: long (nullable = true)



In [22]:
from pyspark.sql.functions import desc, col

num_comments = df.count()
print('number of comments: {}'.format(num_comments))

start_time_0 = timeit.default_timer() 

author_comment_counts = df.groupBy('author').count().sort(desc("count"))
author_comment_counts.show()

print("execution time {}s".format(timeit.default_timer()-start_time_0))



number of comments: 1075
+---------------+-----+
|         author|count|
+---------------+-----+
|      [deleted]|  185|
|   michaelneale|   21|
|       symbiont|   21|
|       mattknox|   19|
|            chu|   16|
|     paulgraham|   15|
|            e40|   13|
|       fnord123|   12|
|          jcage|   10|
|         stesch|    9|
|        beza1e1|    9|
|        bugbear|    9|
|         TronXD|    9|
|         rdtiii|    9|
|       enjahova|    9|
|       JimThome|    8|
| JonathanGCohen|    8|
|      lynxcat73|    8|
|      drewyates|    7|
|theycallmemorty|    7|
+---------------+-----+
only showing top 20 rows

execution time 4.649566684005549s
+-------------+----------+
|       author|sum(score)|
+-------------+----------+
|   paulgraham|        98|
|     mattknox|        77|
|    [deleted]|        58|
|      bugbear|        43|
| michaelneale|        39|
|     dstowell|        38|
|         spez|        36|
|      AaronSw|        30|
|        sempf|        29|
|     JimThome|

In [28]:
start_time_0 = timeit.default_timer() 
df.groupBy('author').sum('score').sort(desc("sum(score)")).show()
print("execution time {}s".format(timeit.default_timer()-start_time_0))

+-------------+----------+
|       author|sum(score)|
+-------------+----------+
|   paulgraham|        98|
|     mattknox|        77|
|    [deleted]|        58|
|      bugbear|        43|
| michaelneale|        39|
|     dstowell|        38|
|         spez|        36|
|      AaronSw|        30|
|     JimThome|        29|
|        sempf|        29|
|    bolinfest|        28|
|       davidw|        27|
|          Zak|        26|
|     symbiont|        25|
|     binladen|        24|
|     enjahova|        23|
|         cg84|        23|
|brendankohler|        21|
|       dylanm|        20|
|     beastboy|        19|
+-------------+----------+
only showing top 20 rows

execution time 11.968498889997136s


In [3]:
# release the cores for another application!
spark_context.stop()
print("Ended Session")

Ended Session
